In [52]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [205]:
mega_millions = ['http://www.lottoreport.com/mm03sales18.htm',
'http://www.lottoreport.com/mm03sales17.htm',
'http://www.lottoreport.com/mm03sales16.htm',
'http://www.lottoreport.com/mm03sales15.htm',
 'http://www.lottoreport.com/mm03salesAug5Dec302014.htm',
 'http://www.lottoreport.com/mm03salesjan3aug12014.htm',
 'http://www.lottoreport.com/mm03salesAug20Dec312013.htm',
 'http://www.lottoreport.com/mm03salesjan1aug162013.htm',
 'http://www.lottoreport.com/mm03salesjuly31dec282012.htm',
 'http://www.lottoreport.com/mm03salesjan3july272012.htm',
 'http://www.lottoreport.com/mm03salesjune7dec302011.htm',
'http://www.lottoreport.com/mm03salesjan18june032011.htm',
 'http://www.lottoreport.com/mm03salesJulyDec2010.htm',
]

In [230]:
power_ball = ['http://www.lottoreport.com/PBSalesbystate2018.htm',
'http://www.lottoreport.com/PBSalesbystate2017.htm',
'http://www.lottoreport.com/PBSalesbystate2016.htm',
              'http://www.lottoreport.com/PBSalesbystate22015.htm',
              'http://www.lottoreport.com/PBSalesbystate12015.htm',
              'http://www.lottoreport.com/PBSalesbystate22014.htm',
              'http://www.lottoreport.com/PBSalesbystate12014.htm',
              'http://www.lottoreport.com/PBSalesbystate22013.htm',
              'http://www.lottoreport.com/PBSalesbystate12013.htm',
              'http://www.lottoreport.com/PBSalesbystate22012.htm',
              'http://www.lottoreport.com/PBSalesbystate12012.htm',
              'http://www.lottoreport.com/PBSalesbystate22011.htm',
              'http://www.lottoreport.com/PBSalesbystate12011.htm',
              'http://www.lottoreport.com/PBSalesbystate22010.htm',
              'http://www.lottoreport.com/PBSalesbystate12010.htm'
]

In [ ]:
# final for mega millions

def build_df(df):
    
    states = [x for x in df[0].values if len(str(x)) == 2]
    
    draw_sales_line = list(df[1].values).index('Draw Sales')
    
    if 'Draw Date' in list(df[0].values):
        draw_date_line = list(df[0].values).index('Draw Date')
        dates = pd.to_datetime([x for x in list(df.loc[draw_date_line]) if '/' in str(x)])
    
        df = df[df[0].isin(states+['State'])]
        df.columns = df.loc[draw_sales_line]
        df = pd.concat([df['State'],df['Draw Sales']],axis=1)
        df.index = df['State'].T.iloc[0]
        df = df.drop(['State'],axis=1)

    else:

        # dates
        state_line = list(df[0].values).index('State')
        state_line_values = list(df.loc[state_line].values)
        dates = [x.replace('* ','').split(' ')[0] for x in state_line_values]
        dates = pd.to_datetime([x for x in dates if '/' in str(x)])    
    
        # move over draw sales by one
        draw_sales_col = df.loc[draw_sales_line].values
        draw_sales_col = ['keep_column']+list(draw_sales_col)
        draw_sales_col = draw_sales_col[:-1]
        df.loc[draw_sales_line] = draw_sales_col
        
        
        df = df[df[0].isin(states+['State','keep_column'])]
        df.columns = df.loc[draw_sales_line]
        df = pd.concat([df['keep_column'],df['Draw Sales']],axis=1)
        df = df[~(df.keep_column.isin(['State','keep_column']))]
        df.index = df['keep_column']
        df = df.drop(['keep_column'],axis=1)
    
    df = df.loc[states]
    df.columns = dates
    df = df.stack().reset_index()
    df.columns = ['state','date','draw_sales']
    
    return df

In [359]:
# final for powerball 

def build_df(df):
    
    states = [x for x in df[0].values if len(str(x)) == 2]
    
    if 'Draw Sales' not in df[1].values:
        
        draw_line = list(df[1].values).index('Draw')
        draw_line_content = df.loc[draw_line]
        draw_line_content = ['Draw Sales' if x == 'Draw' else x for x in draw_line_content]
        df.loc[draw_line] = draw_line_content
        
    draw_sales_line = list(df[1].values).index('Draw Sales')
    
    draw_date_line = list(df[0].values).index('Draw Date')
    dates = pd.to_datetime([x for x in list(df.loc[draw_date_line]) if '/' in str(x)])

    df = df[df[0].isin(states+['State'])]
    df.columns = df.loc[draw_sales_line]
    df = pd.concat([df['State'],df['Draw Sales']],axis=1)

    if len(df['State'].shape) == 2:
        new_index = [x[0] if len(x) == 2 else x for x in df[['State']].values ]

    else:
        new_index = df['State']

    df.index = new_index
    df = df.drop(['State'],axis=1)

    df = df.loc[states]
    df.columns = dates
    df = df.stack().reset_index()
    df.columns = ['state','date','draw_sales']
    
    return df

In [206]:
mm_df = pd.DataFrame()

for site in mega_millions:
    
    site_dfs = pd.read_html(site)

    for i in range(len(site_dfs)):

        df = build_df(site_dfs[i])
        mm_df = pd.concat([mm_df,df],axis=0)

# some had two numbers, taking first one
mm_df['draw_sales'] = mm_df['draw_sales'].apply(lambda x: x.split(' ')[0])

# remove any non-digits
mm_df['draw_sales'] = mm_df.draw_sales.str.replace(r"[^\d]",'')

mm_df['draw_sales'] = pd.to_numeric(mm_df['draw_sales'])

In [338]:
pb_df = pd.DataFrame()

for site in power_ball:
    
    site_dfs = pd.read_html(site)

    for i in range(len(site_dfs)):

        df = build_df(site_dfs[i])
        pb_df = pd.concat([pb_df,df],axis=0)

# some had two numbers, taking first one
pb_df['draw_sales'] = pb_df['draw_sales'].apply(lambda x: x.split(' ')[0])

# remove any non-digits
pb_df['draw_sales'] = pb_df.draw_sales.str.replace(r"[^\d]",'')

pb_df['draw_sales'] = pd.to_numeric(pb_df['draw_sales'])


if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
if path
